In [1]:
import numpy as np
import pandas as pd
import re
import collections
import operator
import simpy
import random

In [2]:
def estatistica_descritiva(data,campo):
    print(f"{campo} Media :",data[campo].mean())
    print(f"{campo} Mediana",data[campo].median())
    print(f"{campo} Moda",list(data[campo].mode()))
    print(f"{campo} Minimo",data[campo].min())
    print(f"{campo} Maximo",data[campo].max())
    print(f"{campo} Amplitude",data[campo].max()-data[campo].min())
    print(f"{campo} Desvio padrão",data[campo].std())
    print(f"{campo} Variacia",data[campo].var())
    print(f"{campo} Coeficinte de variação",data[campo].std()/data[campo].mean())
    print(f"{campo} Assimetria",data[campo].skew())

In [3]:
def tempo_medio (data,trantime=False,breaktime=False):
    
    if trantime:
        produtos_segundos = 0
        for index, trasacao in data.iterrows():
            produtos_segundos += trasacao["TranTime"]/trasacao['ArtNum']
        media_produtos_segundos = produtos_segundos/len(data.index)
        return(media_produtos_segundos)
    
    elif breaktime:
        produtos_segundos = 0
        for index, trasacao in data.iterrows():
            produtos_segundos += (trasacao["BreakTime"])/trasacao['ArtNum']
        media_produtos_segundos=produtos_segundos/len(data.index)
        return(media_produtos_segundos)
        
    else:
        produtos_segundos = 0
        for index, trasacao in data.iterrows():
            produtos_segundos +=np.add(trasacao["TranTime"],trasacao["BreakTime"])/trasacao['ArtNum']
        media_produtos_segundos=produtos_segundos/len(data.index)
        return(media_produtos_segundos)

In [4]:
trasacao = pd.read_csv("data/POS_transactions_20190328-20190410.csv")

In [5]:
trasacao.dtypes

WorkstationGroupID      int64
TranID                float64
BeginDateTime          object
EndDateTime            object
OperatorID              int64
TranTime                int64
BreakTime               int64
ArtNum                  int64
TNcash                   bool
TNcard                   bool
Amount                float64
dtype: object

In [6]:
trasacao_workstation_service = trasacao.query('WorkstationGroupID == 1')

In [7]:
trasacao_workstation_service_date = trasacao_workstation_service.loc[trasacao_workstation_service.BeginDateTime.str.contains('2019-04-03T10|2019-04-03T11|2019-04-03T12', flags=re.I, regex=True)]

## Removendo as linha com TranTime igual a zero

In [8]:
for index, transacao in trasacao_workstation_service_date.iterrows():
    if transacao["TranTime"]==0:
        trasacao_workstation_service_date=trasacao_workstation_service_date.drop([index])

In [9]:
trasacao_workstation_service_date.describe()

,WorkstationGroupID,TranID,OperatorID,TranTime,BreakTime,ArtNum,Amount
count,510.0,5.100000e+02,510.000000,510.000000,510.000000,510.000000,510.000000
mean,1.0,6.874224e+13,220.229412,50.854902,33.194118,15.147059,62.173235
std,0.0,7.988460e+13,94.311403,40.053924,71.336329,14.968990,78.679271
min,1.0,1.904031e+11,105.000000,4.000000,6.000000,1.000000,2.170000
25%,1.0,1.904031e+13,128.000000,24.000000,15.000000,6.000000,21.337500
50%,1.0,1.904031e+13,301.000000,41.000000,21.000000,11.000000,41.380000
75%,1.0,1.904031e+14,311.000000,67.000000,32.000000,19.000000,72.785000
max,1.0,1.904031e+14,361.000000,309.000000,1147.000000,123.000000,811.750000


## Estatistica descritiva todas as formas de pagamentos

### Estatistica descritiva campo TranTime

In [10]:
estatistica_descritiva(trasacao_workstation_service_date,"TranTime")

TranTime Media : 50.85490196078431
TranTime Mediana 41.0
TranTime Moda [26, 31]
TranTime Minimo 4
TranTime Maximo 309
TranTime Amplitude 305
TranTime Desvio padrão 40.05392393613791
TranTime Variacia 1604.3168226819214
TranTime Coeficinte de variação 0.7876118602494732
TranTime Assimetria 2.0457119278440214


### Estatistica descritiva campo Breaktime

In [11]:
estatistica_descritiva(trasacao_workstation_service_date,"BreakTime")


BreakTime Media : 33.194117647058825
BreakTime Mediana 21.0
BreakTime Moda [17]
BreakTime Minimo 6
BreakTime Maximo 1147
BreakTime Amplitude 1141
BreakTime Desvio padrão 71.33632925114235
BreakTime Variacia 5088.871871027388
BreakTime Coeficinte de variação 2.149065385910721
BreakTime Assimetria 12.041105864454947


### Estatistica Descritiva campo numeros de artigos

In [12]:
estatistica_descritiva(trasacao_workstation_service_date,"ArtNum")

ArtNum Media : 15.147058823529411
ArtNum Mediana 11.0
ArtNum Moda [3]
ArtNum Minimo 1
ArtNum Maximo 123
ArtNum Amplitude 122
ArtNum Desvio padrão 14.968990250918493
ArtNum Variacia 224.0706691320929
ArtNum Coeficinte de variação 0.9882440165654928
ArtNum Assimetria 2.6869050219189594


### Estatistica Descritiva campo TNcash e TNcard

In [13]:
trasacao_workstation_service_date["TNcash"].describe()

count      510
unique       2
top       True
freq       335
Name: TNcash, dtype: object

### Calculando o tempo medio de item por segundo



In [14]:
tempo_medio(trasacao_workstation_service_date)

9.205502027971132

###  Calculando o tempo medio de item por segundo considerando somente o tempo de trasação

In [15]:
tempo_medio(trasacao_workstation_service_date,trantime=True)

4.669640765279946

###  Calculando o tempo medio de item por segundo considerando somente o tempo de empacotamento

In [16]:
tempo_medio(trasacao_workstation_service_date,breaktime=True)

4.53586126269119

## Estatistica descritiva considerando apenas compras no dinheiro


In [17]:
trasacao_workstation_service_date_tncash = trasacao_workstation_service_date.query('TNcash == True')

### Estatistica descritiva campo TranTime

In [18]:
estatistica_descritiva(trasacao_workstation_service_date_tncash,"TranTime")

TranTime Media : 44.208955223880594
TranTime Mediana 35.0
TranTime Moda [11, 26, 28]
TranTime Minimo 4
TranTime Maximo 309
TranTime Amplitude 305
TranTime Desvio padrão 37.53096019896204
TranTime Variacia 1408.572973456073
TranTime Coeficinte de variação 0.8489447445410051
TranTime Assimetria 2.6301824423971367


### Estatistica descritiva campo BreakTime

In [19]:
estatistica_descritiva(trasacao_workstation_service_date_tncash,"BreakTime")

BreakTime Media : 33.56417910447761
BreakTime Mediana 22.0
BreakTime Moda [17, 19]
BreakTime Minimo 8
BreakTime Maximo 940
BreakTime Amplitude 932
BreakTime Desvio padrão 57.94443885705378
BreakTime Variacia 3357.557994458844
BreakTime Coeficinte de variação 1.7263773583344912
BreakTime Assimetria 12.235668470386164


### Estatistica descritiva campo ArtNum

In [20]:
estatistica_descritiva(trasacao_workstation_service_date_tncash,"ArtNum")

ArtNum Media : 13.170149253731344
ArtNum Mediana 10.0
ArtNum Moda [3]
ArtNum Minimo 1
ArtNum Maximo 123
ArtNum Amplitude 122
ArtNum Desvio padrão 13.458663871692076
ArtNum Variacia 181.13563321118957
ArtNum Coeficinte de variação 1.0219067082993756
ArtNum Assimetria 3.526173671756291


### Calculando o tempo medio de item por segundo

In [21]:
tempo_medio(trasacao_workstation_service_date_tncash)

9.980679204243227

###  Calculando o tempo medio de item por segundo considerando somente o tempo de trasação

In [22]:
tempo_medio(trasacao_workstation_service_date_tncash,trantime=True)

4.537310756315868

###  Calculando o tempo medio de item por segundo considerando somente o tempo de empacotamento

In [23]:
tempo_medio(trasacao_workstation_service_date_tncash,breaktime=True)

5.4433684479273605

## Estatistica descritiva considerando apenas no cartão de credito

In [24]:
trasacao_workstation_service_date_tncard = trasacao_workstation_service_date.query('TNcard == True')

### Estatistica descritiva campo TranTime

In [25]:
estatistica_descritiva(trasacao_workstation_service_date_tncard,"TranTime")

TranTime Media : 64.33714285714285
TranTime Mediana 52.0
TranTime Moda [31]
TranTime Minimo 13
TranTime Maximo 251
TranTime Amplitude 238
TranTime Desvio padrão 41.99487575277451
TranTime Variacia 1763.5695894909684
TranTime Coeficinte de variação 0.6527314376708002
TranTime Assimetria 1.3877977720260803


### Estatistica descritiva campo BreakTime

In [26]:
estatistica_descritiva(trasacao_workstation_service_date_tncard,"BreakTime")

BreakTime Media : 32.48
BreakTime Mediana 16.0
BreakTime Moda [14]
BreakTime Minimo 6
BreakTime Maximo 1147
BreakTime Amplitude 1141
BreakTime Desvio padrão 91.86445787511977
BreakTime Variacia 8439.078620689656
BreakTime Coeficinte de variação 2.828339220293097
BreakTime Assimetria 10.706897789257468


### Estatistica descritiva campo ArtNum

In [27]:
estatistica_descritiva(trasacao_workstation_service_date_tncard,"ArtNum")

ArtNum Media : 18.925714285714285
ArtNum Mediana 13.0
ArtNum Moda [3]
ArtNum Minimo 1
ArtNum Maximo 98
ArtNum Amplitude 97
ArtNum Desvio padrão 17.01622523306112
ArtNum Variacia 289.55192118226597
ArtNum Coeficinte de variação 0.8991061038000291
ArtNum Assimetria 1.7948384626166136


### Calculando o tempo medio de item por segundo

In [28]:
tempo_medio(trasacao_workstation_service_date_tncard)

7.877840225663656

### Calculando o tempo medio de item por transação

In [29]:
tempo_medio(trasacao_workstation_service_date_tncard,trantime=True)

5.0663667686371365

### Calculando o tempo medio item por tempo de empacotamento

In [30]:
tempo_medio(trasacao_workstation_service_date_tncard,breaktime=True)

2.8114734570265214

In [31]:
frequencia = collections.Counter(trasacao_workstation_service_date["ArtNum"])

In [32]:
frequencia_ordenada = sorted(frequencia.items(), key=operator.itemgetter(1), reverse=True)

In [33]:
frequencia_porcentagem = {}

In [34]:
frequencia_f = []
for frequencia in frequencia_ordenada:
    porcentagem = (frequencia[1]/len(trasacao_workstation_service_date.index))*100
    frequencia_f.append((frequencia[0],porcentagem))


In [35]:
frequencia_dividida_tamanho = {"qtd_pequena": [], "qtd_media": [], "qtd_grande": []}
soma_frequencia_pequena = 0
soma_frequencia_media = 0
soma_frequencia_grande = 0
for frequencia in frequencia_f:
    if frequencia[0] <= 10 :
        frequencia_dividida_tamanho["qtd_pequena"].append(frequencia[0])
        soma_frequencia_pequena += frequencia[1] 
    elif (frequencia[0] > 10) & (frequencia[0]<=30):
        frequencia_dividida_tamanho["qtd_media"].append(frequencia[0])
        soma_frequencia_media += frequencia[1]
    else :
        frequencia_dividida_tamanho["qtd_grande"].append(frequencia[0])
        soma_frequencia_grande += frequencia[1]
frequencia_dividida_tamanho["qtd_pequena"] = soma_frequencia_pequena
frequencia_dividida_tamanho["qtd_media"] = soma_frequencia_media
frequencia_dividida_tamanho["qtd_grande"] = soma_frequencia_grande
print(frequencia_dividida_tamanho)

{'qtd_pequena': 49.2156862745098, 'qtd_media': 39.607843137254896, 'qtd_grande': 11.176470588235286}


# simulação simpy

In [36]:
class cliente:
  def __init__(self,id,n_itens):
    self.id_cliente = id
    self.n_itens = n_itens 
    self.tempo_inicial = None
    self.tempo_final = None
  
  
  def set_tempo_incial(self,tempo_incial):
    self.tempo_incial = tempo_inicial
  
  
  def set_tempo_final(self,tempo_final):
    self.tempo_final = tempo_final

In [37]:
class itens:
  def __init__(self):
    self.clientes=[]
    for i in range(1,80):
      self.clientes.append(random.randrange(1,20))
    for i in range(80,101):
      self.clientes.append(random.randrange(20,100))
Itens = itens()


In [38]:
def criação_de_filas_linear():
  index=0
  filas=[[],[],[],[]]
  clientes=[]
  #print(clientes)
  for cliente in Itens.clientes:
    if index == 4:
      index=0
    filas[index].append((cliente))
    index+=1
  for i in filas:
    print ("fila->",i,"tamanho da fila->",len(i))
  return (filas)

In [39]:
def criação_de_filas_com_prioridade():
  index=0
  filas=[[],[],[],[]]
  for cliente in Itens.clientes:
    if cliente <= 10:
      filas[0].append(cliente)
    if cliente >= 11 and cliente <= 30:
      filas[1].append(cliente)  
    if cliente >= 31 and cliente <=60:
      filas[2].append(cliente)
    if cliente > 60:
      filas[3].append(cliente)
  for i in filas:
    print ("fila->",i,"tamanho da fila->",len(i))
  return (filas)


In [40]:
def menor_peso_fila(filas):
    menor_peso=9999999999
    index_vez=0
    index=0
    for i in filas:
      if sum(i)>0:
        if (sum(i)) < menor_peso :
          menor_peso=sum(i)
          index_vez=index
        index+=1
      else:
        index_vez=index
        return (index_vez)
        
    return (index_vez)

In [41]:
def criação_de_filas():
  index=0
  filas=[[],[],[],[]]
  #print(clientes)
  for cliente in Itens.clientes:
    vez=(menor_peso_fila(filas))
    #print(index)
    filas[vez].append((cliente))
    index+=1
  for i in filas:
    print ("fila->",i,"tamanho da fila->",len(i))
  return (filas)#cada index representa um caixa 

In [42]:

class caixa:
  def __init__(self,numero_caixa,env,fila):
    self.numero_caixa = numero_caixa
    self.env = env
    self.fila = fila
    self.caixas=[]
    self.caixa_disponivel = self.env.event()
    self.env.process(self.caixa_aberto())
    self.env.process(self.transacao())
    self.clientes = []
    self.clientes_finalizados = []
    
    
  def set_embalador(self):
    self.embalador = True
  
  def set_caixas(self,caixas):
    self.caixas=caixas


  def transacao(self):
    index=0
    yield   self.caixa_disponivel
    #print('eu executo--->',len(self.clientes))
    Clientes=self.clientes.copy()
    #print(Clientes)
    for cliente_atual in Clientes:
      n_itens=cliente_atual.n_itens
      if len(self.clientes)>0:
        print("inicia cliente ##########################################"
              ,"Cliente atual",cliente_atual.id_cliente,"QTD iten do cliente",
              cliente_atual.n_itens,"Caixa",self.numero_caixa,"Tempo atual",self.env.now )
        #print('eu executo--> numero de itens',cliente_atual.n_itens)
        cliente_atual.tempo_inicial = self.env.now
        #print("o cliente  ",cliente_atual.id_cliente ,"está sendo atendido no tempo ", self.env.now,"caixa ", self.numero_caixa,'\n\n\#####################')
        while cliente_atual.n_itens > 0 :
          cliente_atual.n_itens = cliente_atual.n_itens-1
          yield   self.env.timeout( 1 )#tempo referente a trasação
        print("o cliente Saiu############################################### "," o cliente foi atendido o caixa",self.numero_caixa ,"está vazio","tempo atual",self.env.now,"\n\n\n\n")
        cliente_atual.set_tempo_final(self.env.now)
        cliente_atual.n_itens=n_itens
        self.clientes_finalizados.append(cliente_atual)
        #print('------------->id',cliente_atual.id_cliente)
        #print('cliente atual --->',cliente_atual.id_cliente)
        self.clientes.remove(cliente_atual)
    print("=========<>==================Acabou a fila do caixa ",self.numero_caixa,"\n\n\n\n")
    self.env.process(self.busca_cliente_outras_filas())



  
  
  def caixa_aberto(self):
    self.criar_fila()
    yield   self.env.timeout(2)
    self.caixa_disponivel.succeed()



  def busca_cliente_outras_filas(self):
      for caixa in self.caixas:
        
        if len(caixa.clientes)>1:
          print("caixa",self.numero_caixa,'--------------->>>Procurando cliente em outra fila, buscando no caixa',caixa.numero_caixa,"tamanho da fila outro caixa",len(caixa.clientes),self.env.now,"\n\n\n")
          self.clientes.append(caixa.clientes[-1])
          #print(caixa.clientes[-1].id_cliente,caixa.numero_caixa,"tamanho da fila",len(caixa.clientes))
          caixa.clientes.remove(caixa.clientes[-1])
          yield   self.env.timeout(10)
          self.env.process(self.transacao())
          break


  
  def criar_fila(self):
    index=0
    for i in self.fila:
      #print(i)
      c=cliente(index,i)
      self.clientes.append(c)
      index+=1
  
  def media_tempo_caixa(self):
    soma=0
    for cliente in self.clientes_finalizados:
      soma=soma+(cliente.tempo_final-cliente.tempo_inicial)
    return (soma/len(self.clientes_finalizados))

## simulações considerando o tempo de transação e embalagem

### simulação considerando pesos na fila

In [43]:
def main_fila_com_peso():
  clientes=[]
  env = simpy.Environment()
  filas = criação_de_filas()
  caixa1=caixa(1,env,filas[0])
  caixa2=caixa(2,env,filas[1])
  caixa3=caixa(3,env,filas[2])
  caixa4=caixa(4,env,filas[3])
  caixa1.set_caixas([caixa2,caixa3,caixa4])
  caixa2.set_caixas([caixa1,caixa3,caixa4])
  caixa3.set_caixas([caixa1,caixa2,caixa4])
  caixa4.set_caixas([caixa1,caixa2,caixa3])
  env.run()
  print("Caixa 1 media de tempo",round((caixa1.media_tempo_caixa()*tempo_medio(trasacao_workstation_service_date))/60,2),"minutos","Cliente atendidos",len(caixa1.clientes_finalizados),"Tempo de simulação caixa 1",round((caixa1.clientes_finalizados[-1].tempo_final*tempo_medio(trasacao_workstation_service_date))/60,2), "minutos")
  print("Caixa 2 media de tempo",round((caixa2.media_tempo_caixa()*tempo_medio(trasacao_workstation_service_date))/60,2),"minutos","Cliente atendidos",len(caixa2.clientes_finalizados),"Tempo de simulação caixa 2",round((caixa2.clientes_finalizados[-1].tempo_final*tempo_medio(trasacao_workstation_service_date))/60,2), "minutos")
  print("Caixa 3 media de tempo",round((caixa3.media_tempo_caixa()*tempo_medio(trasacao_workstation_service_date))/60,2),"minutos","Cliente atendidos",len(caixa3.clientes_finalizados),"Tempo de simulação caixa 3",round((caixa3.clientes_finalizados[-1].tempo_final*tempo_medio(trasacao_workstation_service_date))/60,2), "minutos")
  print("Caixa 4 media de tempo",round((caixa4.media_tempo_caixa()*tempo_medio(trasacao_workstation_service_date))/60,2),"minutos","Cliente atendidos",len(caixa4.clientes_finalizados),"Tempo de simulação caixa 4",round((caixa4.clientes_finalizados[-1].tempo_final*tempo_medio(trasacao_workstation_service_date))/60,2), "minutos")
  print("media de tempo dos caixas",round((((caixa1.media_tempo_caixa()+caixa2.media_tempo_caixa()+caixa3.media_tempo_caixa()+caixa4.media_tempo_caixa())/4)*tempo_medio(trasacao_workstation_service_date))/60,2),"minutos")
 
main_fila_com_peso()

fila-> [9, 10, 12, 17, 12, 14, 1, 3, 14, 2, 16, 4, 2, 18, 3, 6, 1, 14, 1, 5, 5, 15, 9, 8, 27, 39, 48, 31, 98, 46, 34] tamanho da fila-> 31
fila-> [11, 16, 15, 3, 4, 2, 1, 13, 17, 6, 15, 13, 14, 11, 7, 19, 11, 13, 6, 58, 89, 32, 60, 89] tamanho da fila-> 24
fila-> [13, 19, 1, 18, 1, 11, 8, 10, 19, 17, 14, 14, 8, 8, 8, 3, 14, 19, 79, 94, 67, 25, 44] tamanho da fila-> 23
fila-> [17, 6, 10, 17, 16, 11, 14, 15, 6, 3, 1, 15, 14, 18, 11, 13, 8, 17, 83, 99, 42, 79] tamanho da fila-> 22
inicia cliente ########################################## Cliente atual 0 QTD iten do cliente 9 Caixa 1 Tempo atual 2
inicia cliente ########################################## Cliente atual 0 QTD iten do cliente 11 Caixa 2 Tempo atual 2
inicia cliente ########################################## Cliente atual 0 QTD iten do cliente 13 Caixa 3 Tempo atual 2
inicia cliente ########################################## Cliente atual 0 QTD iten do cliente 17 Caixa 4 Tempo atual 2
o cliente Saiu############################

Caixa 1 media de tempo 2.59 minutos Cliente atendidos 31 Tempo de simulação caixa 1 80.7 minutos
Caixa 2 media de tempo 3.36 minutos Cliente atendidos 24 Tempo de simulação caixa 2 80.85 minutos
Caixa 3 media de tempo 3.43 minutos Cliente atendidos 23 Tempo de simulação caixa 3 79.17 minutos
Caixa 4 media de tempo 3.59 minutos Cliente atendidos 22 Tempo de simulação caixa 4 79.32 minutos
media de tempo dos caixas 3.24 minutos


### Simulação considerando filas com o mesmo tamanho

In [44]:
def main_filas_distribuidas_iguais():
  clientes=[]
  env = simpy.Environment()
  filas = criação_de_filas_linear()
  caixa1=caixa(1,env,filas[0])
  caixa2=caixa(2,env,filas[1])
  caixa3=caixa(3,env,filas[2])
  caixa4=caixa(4,env,filas[3])
  caixa1.set_caixas([caixa2,caixa2,caixa4])
  caixa2.set_caixas([caixa1,caixa3,caixa4])
  caixa3.set_caixas([caixa1,caixa2,caixa4])
  caixa4.set_caixas([caixa1,caixa2,caixa3])
  env.run()
  print("Caixa 1 media de tempo",round((caixa1.media_tempo_caixa()*tempo_medio(trasacao_workstation_service_date))/60,2),"minutos","Cliente atendidos",len(caixa1.clientes_finalizados),"Tempo de simulação caixa 1",round((caixa1.clientes_finalizados[-1].tempo_final*tempo_medio(trasacao_workstation_service_date))/60,2), "minutos")
  print("Caixa 2 media de tempo",round((caixa2.media_tempo_caixa()*tempo_medio(trasacao_workstation_service_date))/60,2),"minutos","Cliente atendidos",len(caixa2.clientes_finalizados),"Tempo de simulação caixa 2",round((caixa2.clientes_finalizados[-1].tempo_final*tempo_medio(trasacao_workstation_service_date))/60,2), "minutos")
  print("Caixa 3 media de tempo",round((caixa3.media_tempo_caixa()*tempo_medio(trasacao_workstation_service_date))/60,2),"minutos","Cliente atendidos",len(caixa3.clientes_finalizados),"Tempo de simulação caixa 3",round((caixa3.clientes_finalizados[-1].tempo_final*tempo_medio(trasacao_workstation_service_date))/60,2), "minutos")
  print("Caixa 4 media de tempo",round((caixa4.media_tempo_caixa()*tempo_medio(trasacao_workstation_service_date))/60,2),"minutos","Cliente atendidos",len(caixa4.clientes_finalizados),"Tempo de simulação caixa 4",round((caixa4.clientes_finalizados[-1].tempo_final*tempo_medio(trasacao_workstation_service_date))/60,2), "minutos")
  print("media de tempo dos caixas",round((((caixa1.media_tempo_caixa()+caixa2.media_tempo_caixa()+caixa3.media_tempo_caixa()+caixa4.media_tempo_caixa())/4)*tempo_medio(trasacao_workstation_service_date))/60,2),"minutos")
main_filas_distribuidas_iguais()

fila-> [9, 10, 12, 1, 4, 1, 14, 10, 14, 15, 13, 2, 15, 14, 1, 19, 8, 15, 13, 6, 27, 89, 31, 67, 46] tamanho da fila-> 25
fila-> [11, 16, 10, 18, 12, 1, 8, 1, 19, 2, 6, 1, 14, 3, 14, 8, 11, 3, 9, 8, 79, 48, 32, 42, 25] tamanho da fila-> 25
fila-> [13, 19, 15, 17, 2, 13, 17, 3, 6, 16, 4, 18, 11, 6, 8, 1, 5, 14, 19, 17, 83, 94, 98, 89, 44] tamanho da fila-> 25
fila-> [17, 6, 17, 3, 16, 11, 11, 14, 15, 17, 3, 14, 14, 7, 18, 5, 11, 13, 8, 58, 39, 99, 60, 79, 34] tamanho da fila-> 25
inicia cliente ########################################## Cliente atual 0 QTD iten do cliente 9 Caixa 1 Tempo atual 2
inicia cliente ########################################## Cliente atual 0 QTD iten do cliente 11 Caixa 2 Tempo atual 2
inicia cliente ########################################## Cliente atual 0 QTD iten do cliente 13 Caixa 3 Tempo atual 2
inicia cliente ########################################## Cliente atual 0 QTD iten do cliente 17 Caixa 4 Tempo atual 2
o cliente Saiu############################

inicia cliente ########################################## Cliente atual 20 QTD iten do cliente 39 Caixa 4 Tempo atual 280
o cliente Saiu###############################################   o cliente foi atendido o caixa 2 está vazio tempo atual 304 




inicia cliente ########################################## Cliente atual 22 QTD iten do cliente 32 Caixa 2 Tempo atual 304
o cliente Saiu###############################################   o cliente foi atendido o caixa 3 está vazio tempo atual 309 




inicia cliente ########################################## Cliente atual 21 QTD iten do cliente 94 Caixa 3 Tempo atual 309
o cliente Saiu###############################################   o cliente foi atendido o caixa 1 está vazio tempo atual 314 




inicia cliente ########################################## Cliente atual 22 QTD iten do cliente 31 Caixa 1 Tempo atual 314
o cliente Saiu###############################################   o cliente foi atendido o caixa 4 está vazio tempo atual 319 


### simulação considerando filas com prioridade (Padrão) 

In [45]:
def main_filas_com_prioridade():
  clientes=[]
  env = simpy.Environment()
  filas = criação_de_filas_com_prioridade()
  caixa1=caixa(1,env,filas[0])
  caixa2=caixa(2,env,filas[1])
  caixa3=caixa(3,env,filas[2])
  caixa4=caixa(4,env,filas[3])
  caixa1.set_caixas([caixa2,caixa2,caixa4])
  caixa2.set_caixas([caixa1,caixa3,caixa4])
  caixa3.set_caixas([caixa1,caixa2,caixa4])
  caixa4.set_caixas([caixa1,caixa2,caixa3])
  env.run()
  print("Caixa 1 media de tempo",round((caixa1.media_tempo_caixa()*tempo_medio(trasacao_workstation_service_date))/60,2),"minutos","Cliente atendidos",len(caixa1.clientes_finalizados),"Tempo de simulação caixa 1",round((caixa1.clientes_finalizados[-1].tempo_final*tempo_medio(trasacao_workstation_service_date))/60,2), "minutos")
  print("Caixa 2 media de tempo",round((caixa2.media_tempo_caixa()*tempo_medio(trasacao_workstation_service_date))/60,2),"minutos","Cliente atendidos",len(caixa2.clientes_finalizados),"Tempo de simulação caixa 2",round((caixa2.clientes_finalizados[-1].tempo_final*tempo_medio(trasacao_workstation_service_date))/60,2), "minutos")
  print("Caixa 3 media de tempo",round((caixa3.media_tempo_caixa()*tempo_medio(trasacao_workstation_service_date))/60,2),"minutos","Cliente atendidos",len(caixa3.clientes_finalizados),"Tempo de simulação caixa 3",round((caixa3.clientes_finalizados[-1].tempo_final*tempo_medio(trasacao_workstation_service_date))/60,2), "minutos")
  print("Caixa 4 media de tempo",round((caixa4.media_tempo_caixa()*tempo_medio(trasacao_workstation_service_date))/60,2),"minutos","Cliente atendidos",len(caixa4.clientes_finalizados),"Tempo de simulação caixa 4",round((caixa4.clientes_finalizados[-1].tempo_final*tempo_medio(trasacao_workstation_service_date))/60,2), "minutos")
  tempo_medio_filas_prioridade_padrao = round((((caixa1.media_tempo_caixa()+caixa2.media_tempo_caixa()+caixa3.media_tempo_caixa()+caixa4.media_tempo_caixa())/4)*tempo_medio(trasacao_workstation_service_date))/60,2)
  print("media de tempo dos caixas",tempo_medio_filas_prioridade_padrao,"minutos")
  return(tempo_medio_filas_prioridade_padrao)
  
tempo_medio_filas_prioridade_padrao=main_filas_com_prioridade()

fila-> [9, 10, 6, 10, 1, 3, 4, 2, 1, 1, 8, 10, 1, 3, 6, 2, 6, 4, 3, 2, 1, 3, 6, 7, 1, 8, 8, 1, 5, 8, 5, 3, 9, 8, 6, 8] tamanho da fila-> 36
fila-> [11, 13, 17, 16, 19, 12, 15, 17, 18, 17, 12, 16, 13, 11, 14, 17, 11, 14, 14, 19, 15, 15, 16, 17, 13, 18, 14, 15, 14, 11, 14, 14, 14, 18, 19, 11, 11, 15, 14, 13, 13, 19, 17, 27, 25] tamanho da fila-> 45
fila-> [58, 39, 48, 31, 32, 60, 42, 46, 44, 34] tamanho da fila-> 10
fila-> [79, 83, 89, 94, 99, 98, 67, 89, 79] tamanho da fila-> 9
inicia cliente ########################################## Cliente atual 0 QTD iten do cliente 9 Caixa 1 Tempo atual 2
inicia cliente ########################################## Cliente atual 0 QTD iten do cliente 11 Caixa 2 Tempo atual 2
inicia cliente ########################################## Cliente atual 0 QTD iten do cliente 58 Caixa 3 Tempo atual 2
inicia cliente ########################################## Cliente atual 0 QTD iten do cliente 79 Caixa 4 Tempo atual 2
o cliente Saiu#############################

inicia cliente ########################################## Cliente atual 4 QTD iten do cliente 99 Caixa 4 Tempo atual 347
o cliente Saiu###############################################   o cliente foi atendido o caixa 1 está vazio tempo atual 355 




=========<>==================Acabou a fila do caixa  1 




caixa 1 --------------->>>Procurando cliente em outra fila, buscando no caixa 2 tamanho da fila outro caixa 16 355 



o cliente Saiu###############################################   o cliente foi atendido o caixa 3 está vazio tempo atual 358 




inicia cliente ########################################## Cliente atual 8 QTD iten do cliente 44 Caixa 3 Tempo atual 358
o cliente Saiu###############################################   o cliente foi atendido o caixa 2 está vazio tempo atual 361 




inicia cliente ########################################## Cliente atual 24 QTD iten do cliente 13 Caixa 2 Tempo atual 361
inicia cliente ########################################## Cliente atua

Caixa 4 media de tempo 13.86 minutos Cliente atendidos 6 Tempo de simulação caixa 4 83.46 minutos
media de tempo dos caixas 6.04 minutos


In [46]:
def porcetagem_melhorar(tempo,tempo_medio_filas_prioridade_padrao):
    porcentagem_atual=(tempo*100)/tempo_medio_filas_prioridade_padrao
    porcentagem=100-porcentagem_atual
    return(porcentagem)
    

## simulação com embalador (desconsiderar break time)

In [47]:
def main_com_embalador():
  clientes=[]
  env = simpy.Environment()
  filas = criação_de_filas_com_prioridade()
  caixa1=caixa(1,env,filas[0])
  caixa2=caixa(2,env,filas[1])
  caixa3=caixa(3,env,filas[2])
  caixa4=caixa(4,env,filas[3])
  caixa1.set_caixas([caixa2,caixa2,caixa4])
  caixa2.set_caixas([caixa1,caixa3,caixa4])
  caixa3.set_caixas([caixa1,caixa2,caixa4])
  caixa4.set_caixas([caixa1,caixa2,caixa3])
  env.run()
  print("Caixa 1 media de tempo",round((caixa1.media_tempo_caixa()*tempo_medio(trasacao_workstation_service_date,trantime=True))/60,2),"minutos","Cliente atendidos",len(caixa1.clientes_finalizados),"Tempo de simulação caixa 1",round((caixa1.clientes_finalizados[-1].tempo_final*tempo_medio(trasacao_workstation_service_date,trantime=True))/60,2), "minutos")
  print("Caixa 2 media de tempo",round((caixa2.media_tempo_caixa()*tempo_medio(trasacao_workstation_service_date,trantime=True))/60,2),"minutos","Cliente atendidos",len(caixa2.clientes_finalizados),"Tempo de simulação caixa 2",round((caixa2.clientes_finalizados[-1].tempo_final*tempo_medio(trasacao_workstation_service_date,trantime=True))/60,2), "minutos")
  print("Caixa 3 media de tempo",round((caixa3.media_tempo_caixa()*tempo_medio(trasacao_workstation_service_date,trantime=True))/60,2),"minutos","Cliente atendidos",len(caixa3.clientes_finalizados),"Tempo de simulação caixa 3",round((caixa3.clientes_finalizados[-1].tempo_final*tempo_medio(trasacao_workstation_service_date,trantime=True))/60,2), "minutos")
  print("Caixa 4 media de tempo",round((caixa4.media_tempo_caixa()*tempo_medio(trasacao_workstation_service_date,trantime=True))/60,2),"minutos","Cliente atendidos",len(caixa4.clientes_finalizados),"Tempo de simulação caixa 4",round((caixa4.clientes_finalizados[-1].tempo_final*tempo_medio(trasacao_workstation_service_date,trantime=True))/60,2), "minutos")
  tempo_medio_filas_prioridade_embalador = round((((caixa1.media_tempo_caixa()+caixa2.media_tempo_caixa()+caixa3.media_tempo_caixa()+caixa4.media_tempo_caixa())/4)*tempo_medio(trasacao_workstation_service_date,trantime=True))/60,2)
  print("media de tempo dos caixas",tempo_medio_filas_prioridade_embalador,"minutos")
  print("a simulação com embalador é :",round(porcetagem_melhorar(tempo_medio_filas_prioridade_embalador,tempo_medio_filas_prioridade_padrao),2),"% mais eficiente em referencia a simulaçao utilizando a media do tempo total")
main_com_embalador()

fila-> [9, 10, 6, 10, 1, 3, 4, 2, 1, 1, 8, 10, 1, 3, 6, 2, 6, 4, 3, 2, 1, 3, 6, 7, 1, 8, 8, 1, 5, 8, 5, 3, 9, 8, 6, 8] tamanho da fila-> 36
fila-> [11, 13, 17, 16, 19, 12, 15, 17, 18, 17, 12, 16, 13, 11, 14, 17, 11, 14, 14, 19, 15, 15, 16, 17, 13, 18, 14, 15, 14, 11, 14, 14, 14, 18, 19, 11, 11, 15, 14, 13, 13, 19, 17, 27, 25] tamanho da fila-> 45
fila-> [58, 39, 48, 31, 32, 60, 42, 46, 44, 34] tamanho da fila-> 10
fila-> [79, 83, 89, 94, 99, 98, 67, 89, 79] tamanho da fila-> 9
inicia cliente ########################################## Cliente atual 0 QTD iten do cliente 9 Caixa 1 Tempo atual 2
inicia cliente ########################################## Cliente atual 0 QTD iten do cliente 11 Caixa 2 Tempo atual 2
inicia cliente ########################################## Cliente atual 0 QTD iten do cliente 58 Caixa 3 Tempo atual 2
inicia cliente ########################################## Cliente atual 0 QTD iten do cliente 79 Caixa 4 Tempo atual 2
o cliente Saiu#############################

o cliente Saiu###############################################   o cliente foi atendido o caixa 2 está vazio tempo atual 140 




inicia cliente ########################################## Cliente atual 9 QTD iten do cliente 17 Caixa 2 Tempo atual 140
o cliente Saiu###############################################   o cliente foi atendido o caixa 1 está vazio tempo atual 142 




inicia cliente ########################################## Cliente atual 30 QTD iten do cliente 5 Caixa 1 Tempo atual 142
o cliente Saiu###############################################   o cliente foi atendido o caixa 1 está vazio tempo atual 147 




inicia cliente ########################################## Cliente atual 31 QTD iten do cliente 3 Caixa 1 Tempo atual 147
o cliente Saiu###############################################   o cliente foi atendido o caixa 3 está vazio tempo atual 147 




inicia cliente ########################################## Cliente atual 3 QTD iten do cliente 31 Caixa 3 Tempo atual 147


Caixa 1 media de tempo 0.72 minutos Cliente atendidos 48 Tempo de simulação caixa 1 43.82 minutos
Caixa 2 media de tempo 1.32 minutos Cliente atendidos 33 Tempo de simulação caixa 2 44.6 minutos
Caixa 3 media de tempo 3.2 minutos Cliente atendidos 13 Tempo de simulação caixa 3 44.05 minutos
Caixa 4 media de tempo 7.03 minutos Cliente atendidos 6 Tempo de simulação caixa 4 42.34 minutos
media de tempo dos caixas 3.07 minutos
a simulação com embalador é : 49.17 % mais eficiente em referencia a simulaçao utilizando a media do tempo total


## Simulação com cartão 

In [48]:
def main_com_Cartão():
  clientes=[]
  env = simpy.Environment()
  filas = criação_de_filas_com_prioridade()
  caixa1=caixa(1,env,filas[0])
  caixa2=caixa(2,env,filas[1])
  caixa3=caixa(3,env,filas[2])
  caixa4=caixa(4,env,filas[3])
  caixa1.set_caixas([caixa2,caixa2,caixa4])
  caixa2.set_caixas([caixa1,caixa3,caixa4])
  caixa3.set_caixas([caixa1,caixa2,caixa4])
  caixa4.set_caixas([caixa1,caixa2,caixa3])
  env.run()
  print("Caixa 1 media de tempo",round((caixa1.media_tempo_caixa()*tempo_medio(trasacao_workstation_service_date_tncard))/60,2),"minutos","Cliente atendidos",len(caixa1.clientes_finalizados),"Tempo de simulação caixa 1",round((caixa1.clientes_finalizados[-1].tempo_final*tempo_medio(trasacao_workstation_service_date_tncard))/60,2), "minutos")
  print("Caixa 2 media de tempo",round((caixa2.media_tempo_caixa()*tempo_medio(trasacao_workstation_service_date_tncard))/60,2),"minutos","Cliente atendidos",len(caixa2.clientes_finalizados),"Tempo de simulação caixa 2",round((caixa2.clientes_finalizados[-1].tempo_final*tempo_medio(trasacao_workstation_service_date_tncard))/60,2), "minutos")
  print("Caixa 3 media de tempo",round((caixa3.media_tempo_caixa()*tempo_medio(trasacao_workstation_service_date_tncard))/60,2),"minutos","Cliente atendidos",len(caixa3.clientes_finalizados),"Tempo de simulação caixa 3",round((caixa3.clientes_finalizados[-1].tempo_final*tempo_medio(trasacao_workstation_service_date_tncard))/60,2), "minutos")
  print("Caixa 4 media de tempo",round((caixa4.media_tempo_caixa()*tempo_medio(trasacao_workstation_service_date_tncard))/60,2),"minutos","Cliente atendidos",len(caixa4.clientes_finalizados),"Tempo de simulação caixa 4",round((caixa4.clientes_finalizados[-1].tempo_final*tempo_medio(trasacao_workstation_service_date_tncard))/60,2), "minutos")
  tempo_medio_filas_prioridade_cartao = round((((caixa1.media_tempo_caixa()+caixa2.media_tempo_caixa()+caixa3.media_tempo_caixa()+caixa4.media_tempo_caixa())/4)*tempo_medio(trasacao_workstation_service_date_tncard))/60,2)
  print("a simulação com Cartão é :",round(porcetagem_melhorar(tempo_medio_filas_prioridade_cartao,tempo_medio_filas_prioridade_padrao),2),"% mais eficiente em referencia a simulaçao utilizando a media do tempo total")
main_com_Cartão()

fila-> [9, 10, 6, 10, 1, 3, 4, 2, 1, 1, 8, 10, 1, 3, 6, 2, 6, 4, 3, 2, 1, 3, 6, 7, 1, 8, 8, 1, 5, 8, 5, 3, 9, 8, 6, 8] tamanho da fila-> 36
fila-> [11, 13, 17, 16, 19, 12, 15, 17, 18, 17, 12, 16, 13, 11, 14, 17, 11, 14, 14, 19, 15, 15, 16, 17, 13, 18, 14, 15, 14, 11, 14, 14, 14, 18, 19, 11, 11, 15, 14, 13, 13, 19, 17, 27, 25] tamanho da fila-> 45
fila-> [58, 39, 48, 31, 32, 60, 42, 46, 44, 34] tamanho da fila-> 10
fila-> [79, 83, 89, 94, 99, 98, 67, 89, 79] tamanho da fila-> 9
inicia cliente ########################################## Cliente atual 0 QTD iten do cliente 9 Caixa 1 Tempo atual 2
inicia cliente ########################################## Cliente atual 0 QTD iten do cliente 11 Caixa 2 Tempo atual 2
inicia cliente ########################################## Cliente atual 0 QTD iten do cliente 58 Caixa 3 Tempo atual 2
inicia cliente ########################################## Cliente atual 0 QTD iten do cliente 79 Caixa 4 Tempo atual 2
o cliente Saiu#############################

inicia cliente ########################################## Cliente atual 26 QTD iten do cliente 14 Caixa 2 Tempo atual 392
o cliente Saiu###############################################   o cliente foi atendido o caixa 3 está vazio tempo atual 402 




inicia cliente ########################################## Cliente atual 9 QTD iten do cliente 34 Caixa 3 Tempo atual 402
o cliente Saiu###############################################   o cliente foi atendido o caixa 1 está vazio tempo atual 404 




=========<>==================Acabou a fila do caixa  1 




caixa 1 --------------->>>Procurando cliente em outra fila, buscando no caixa 2 tamanho da fila outro caixa 11 404 



o cliente Saiu###############################################   o cliente foi atendido o caixa 2 está vazio tempo atual 406 




inicia cliente ########################################## Cliente atual 27 QTD iten do cliente 15 Caixa 2 Tempo atual 406
inicia cliente ########################################## Cliente atu